In [ ]:
pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.5-cp310-cp310-linux_x86_64.whl size=2746516 sha256=34c52f7945b4293feef4205d566df4d22f016ab3689a39cbcdf8c4875dbc38ef
  Stored in directory: /root/.cache/pip/wheels/18/2b/7f/c852523089e9182b45fca50ff56f49a51eeb6284fd25a66713
Successfully built mpi4py


In [ ]:
pip uninstall opencv-python


Found existing installation: opencv-python 4.8.0.76
Uninstalling opencv-python-4.8.0.76:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/cv2/*
    /usr/local/lib/python3.10/dist-packages/opencv_python-4.8.0.76.dist-info/*
    /usr/local/lib/python3.10/dist-packages/opencv_python.libs/libQt5Core-39545cc7.so.5.15.0
    /usr/local/lib/python3.10/dist-packages/opencv_python.libs/libQt5Gui-a7aedf18.so.5.15.0
    /usr/local/lib/python3.10/dist-packages/opencv_python.libs/libQt5Test-c38a5234.so.5.15.0
    /usr/local/lib/python3.10/dist-packages/opencv_python.libs/libQt5Widgets-e69d94fb.so.5.15.0
    /usr/local/lib/python3.10/dist-packages/opencv_python.libs/libQt5XcbQpa-5b2d853e.so.5.15.0
    /usr/local/lib/python3.10/dist-packages/opencv_python.libs/libX11-xcb-69166bdf.so.1.0.0
    /usr/local/lib/python3.10/dist-packages/opencv_python.libs/libXau-00ec42fe.so.6.0.0
    /usr/local/lib/python3.10/dist-packages/opencv_python.libs/libavcodec-402e4b05.so.59.37.100
    /usr/local/lib/py

In [ ]:
pip uninstall opencv-contrib-python


In [ ]:
pip uninstall opencv-contrib-python-headless

In [ ]:
pip install opencv-contrib-python==4.5.5.62

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls gdrive/MyDrive


In [ ]:
!unzip gdrive/MyDrive/corel.zip

In [ ]:
pip install pywavelets


In [ ]:
from mpi4py import MPI
import cv2
import numpy as np
import time
import os

# Function for shape feature extraction using Laplacian edge detection and Hu Moments
def extract_shape_features(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Laplacian edge detection
    edges = cv2.Laplacian(gray, cv2.CV_64F)

    # Hu Moments
    moments = cv2.HuMoments(cv2.moments(edges)).flatten()

    return moments

# Function for color feature extraction using all channels of YCbCr color space and color histograms
def extract_color_features(image_path):
    image = cv2.imread(image_path)

    # YCbCr color space
    image_ycbcr = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # Separate channels
    y, cb, cr = cv2.split(image_ycbcr)

    # Color histograms
    hist_y = cv2.calcHist([y], [0], None, [256], [0, 256]).flatten()
    hist_cb = cv2.calcHist([cb], [0], None, [256], [0, 256]).flatten()
    hist_cr = cv2.calcHist([cr], [0], None, [256], [0, 256]).flatten()

    # Concatenate histograms
    color_features = np.concatenate((hist_y, hist_cb, hist_cr))

    return color_features

# Function to calculate Euclidean distance between two feature vectors
def euclidean_distance(vec1, vec2):
    return np.linalg.norm(vec1 - vec2)

# MPI Initialization
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
print(size=size)
# Specify the path to the "corel_images" dataset
corel_images_path = '/content/dataset'  # Replace with the actual path

# Master node reads the dataset and broadcasts it to all nodes
if rank == 0:
    dataset = []

    for root, dirs, files in os.walk(corel_images_path):
        for file in files:
            if file.endswith('.jpg'):
                image_path = os.path.join(root, file)
                dataset.append(image_path)
else:
    dataset = None

dataset = comm.bcast(dataset, root=0)

# Each node processes a subset of the dataset
local_dataset = np.array_split(dataset, size)[rank]

# Timing starts
start_time = MPI.Wtime()

# Feature extraction
local_shape_features = []
local_color_features = []

for image_path in local_dataset:
    local_shape_features.append(extract_shape_features(image_path))
    local_color_features.append(extract_color_features(image_path))

# Gather results to the master node
shape_features = comm.gather(local_shape_features, root=0)
color_features = comm.gather(local_color_features, root=0)

# Timing ends
end_time = MPI.Wtime()

# Master node prints the total time
if rank == 0:
    total_time = end_time - start_time
    print("Total time taken: {:.2f} milliseconds".format(total_time))

    # Process the gathered features
    all_shape_features = np.concatenate(shape_features)
    all_color_features = np.concatenate(color_features)

    # Replace this placeholder with the actual path to the query image in the "corel_images" dataset
    query_image_path = '/content/dataset/test_set/beaches/100.jpg'

    query_shape_features = extract_shape_features(query_image_path)
    query_color_features = extract_color_features(query_image_path)

    # Calculate distances and rank images
    distances_shape = [euclidean_distance(query_shape_features, features) for features in all_shape_features]
    distances_color = [euclidean_distance(query_color_features, features) for features in all_color_features]

    # Combine distances from both shape and color features
    combined_distances = np.array(distances_shape) + np.array(distances_color)

    # Retrieve the indices of the sorted distances
    retrieval_order = np.argsort(combined_distances)

    print("Retrieval order:", retrieval_order)
    print("Top 3 retrieved images:", [dataset[i] for i in retrieval_order[:3]])


In [ ]:
import cv2
import numpy as np
import time
import os

# Function for shape feature extraction using Laplacian edge detection and Hu Moments
def extract_shape_features(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Laplacian edge detection
    edges = cv2.Laplacian(gray, cv2.CV_64F)

    # Hu Moments
    moments = cv2.HuMoments(cv2.moments(edges)).flatten()

    return moments

# Function for color feature extraction using all channels of YCbCr color space and color histograms
def extract_color_features(image_path):
    image = cv2.imread(image_path)

    # YCbCr color space
    image_ycbcr = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # Separate channels
    y, cb, cr = cv2.split(image_ycbcr)

    # Color histograms
    hist_y = cv2.calcHist([y], [0], None, [256], [0, 256]).flatten()
    hist_cb = cv2.calcHist([cb], [0], None, [256], [0, 256]).flatten()
    hist_cr = cv2.calcHist([cr], [0], None, [256], [0, 256]).flatten()

    # Concatenate histograms
    color_features = np.concatenate((hist_y, hist_cb, hist_cr))

    return color_features

# Function to calculate Euclidean distance between two feature vectors
def euclidean_distance(vec1, vec2):
    return np.linalg.norm(vec1 - vec2)

# Specify the path to the "corel_images" dataset
corel_images_path = '/content/dataset'  # Replace with the actual path

# Read the dataset
dataset = []
for root, dirs, files in os.walk(corel_images_path):
    for file in files:
        if file.endswith('.jpg'):
            image_path = os.path.join(root, file)
            dataset.append(image_path)

# Timing starts
start_time = time.time()

# Feature extraction
all_shape_features = []
all_color_features = []

for image_path in dataset:
    all_shape_features.append(extract_shape_features(image_path))
    all_color_features.append(extract_color_features(image_path))

# Timing ends
end_time = time.time()

total_time = end_time - start_time
print("Total time taken: {:.2f} seconds".format(total_time))

# Replace this placeholder with the actual path to the query image in the "corel_images" dataset
query_image_path = '/content/dataset/test_set/beaches/100.jpg'

query_shape_features = extract_shape_features(query_image_path)
query_color_features = extract_color_features(query_image_path)

# Calculate distances and rank images
distances_shape = [euclidean_distance(query_shape_features, features) for features in all_shape_features]
distances_color = [euclidean_distance(query_color_features, features) for features in all_color_features]

# Combine distances from both shape and color features
combined_distances = np.array(distances_shape) + np.array(distances_color)

# Retrieve the indices of the sorted distances
retrieval_order = np.argsort(combined_distances)

print("Retrieval order:", retrieval_order)
print("Top 3 retrieved images:", [dataset[i] for i in retrieval_order[:3]])


In [ ]:
from mpi4py import MPI
import cv2
import numpy as np
import time
import os
from multiprocessing import Pool

# Function for shape feature extraction using Laplacian edge detection and Hu Moments
def extract_shape_features(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Laplacian edge detection
    edges = cv2.Laplacian(gray, cv2.CV_64F)

    # Hu Moments
    moments = cv2.HuMoments(cv2.moments(edges)).flatten()

    return moments

# Function for color feature extraction using all channels of YCbCr color space and color histograms
def extract_color_features(image_path):
    image = cv2.imread(image_path)

    # YCbCr color space
    image_ycbcr = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # Separate channels
    y, cb, cr = cv2.split(image_ycbcr)

    # Color histograms
    hist_y = cv2.calcHist([y], [0], None, [256], [0, 256]).flatten()
    hist_cb = cv2.calcHist([cb], [0], None, [256], [0, 256]).flatten()
    hist_cr = cv2.calcHist([cr], [0], None, [256], [0, 256]).flatten()

    # Concatenate histograms
    color_features = np.concatenate((hist_y, hist_cb, hist_cr))

    return color_features

# Function to calculate Euclidean distance between two feature vectors
def euclidean_distance(vec1, vec2):
    return np.linalg.norm(vec1 - vec2)

# MPI Initialization
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Specify the path to the "corel_images" dataset
corel_images_path = '/content/dataset'  # Replace with the actual path

# Master node reads the dataset and broadcasts it to all nodes
if rank == 0:
    dataset = []

    for root, dirs, files in os.walk(corel_images_path):
        for file in files:
            if file.endswith('.jpg'):
                image_path = os.path.join(root, file)
                dataset.append(image_path)
else:
    dataset = None

# Scatter the dataset among processes
local_dataset = comm.scatter(np.array_split(dataset, size), root=0)

# Timing starts
start_time = MPI.Wtime()

# Parallelize feature extraction within each process using multiprocessing.Pool
with Pool() as pool:
    local_shape_features = pool.map(extract_shape_features, local_dataset)
    local_color_features = pool.map(extract_color_features, local_dataset)

# Gather results to the master node
shape_features = comm.gather(local_shape_features, root=0)
color_features = comm.gather(local_color_features, root=0)

# Timing ends
end_time = MPI.Wtime()

# Master node prints the total time
if rank == 0:
    total_time = end_time - start_time
    print("Total time taken: {:.2f} seconds".format(total_time))

    # Process the gathered features
    all_shape_features = np.concatenate(shape_features)
    all_color_features = np.concatenate(color_features)

    # Replace this placeholder with the actual path to the query image in the "corel_images" dataset
    query_image_path = '/content/dataset/test_set/beaches/100.jpg'

    query_shape_features = extract_shape_features(query_image_path)
    query_color_features = extract_color_features(query_image_path)

    # Calculate distances and rank images
    distances_shape = [euclidean_distance(query_shape_features, features) for features in all_shape_features]
    distances_color = [euclidean_distance(query_color_features, features) for features in all_color_features]

    # Combine distances from both shape and color features
    combined_distances = np.array(distances_shape) + np.array(distances_color)

    # Retrieve the indices of the sorted distances
    retrieval_order = np.argsort(combined_distances)

    print("Retrieval order:", retrieval_order)
    print("Top 3 retrieved images:", [dataset[i] for i in retrieval_order[:3]])
